In [1]:
import pickle
import os
import pandas as pd
import sentence_transformers
import numpy as np

2025-02-13 23:44:56.447541: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 23:44:56.508686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-13 23:44:57.638148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from sentence_transformers import SentenceTransformer
device = 'cuda:3'
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2").to(device)

## 0k

### vanilla top_k

In [3]:
ROOT_DIR = '/home/prasoon/multilingual-long-reasoning'
PATH_TO_FOLDER =  ROOT_DIR + '/' + 'prompts_0k'

In [31]:
top_k = 10
separator = {'en': '.', 'ar': '.', 'de': '.', 'es' : '.', 'vi': '.', 'zh' : '。', 'hi': '।'}

for file in os.listdir(PATH_TO_FOLDER):

    language = file.split('.')[0]
    df = pd.read_csv(PATH_TO_FOLDER + '/' + file)
    rag_prompts = []
    for index, row in df.iterrows():
        
        passage = row['prompt'].split('<context>')[-1].split('</context>')[0]
        sentences = passage.strip().split(separator[language])[:-1]
        query = row['prompt'].split('</context>')[-1].split('\n')[2].split('QUESTION:')[-1].strip()
                
        query_embedding = model.encode(query)
        passage_embeddings = model.encode(sentences)
                
        similarity = model.similarity(query_embedding, passage_embeddings)
        indices = np.argsort(similarity[0].tolist())[-top_k:][::-1]
        chosen_sentences = [sentences[index].strip() for index in indices]

        joiner = separator[language] + ' '
        chosen_passage = joiner.join(chosen_sentences)+'.'
        rag_prompt = row['prompt'].split('<context>')[0] + '<context>\n' + chosen_passage + '\n</context>' +  row['prompt'].split('</context>')[-1]
        rag_prompts.append(rag_prompt)

    df['rag_prompt_{}'.format(top_k)] = rag_prompts
    df.to_csv(PATH_TO_FOLDER + '/' + file, index=False)

## 2k

### vanilla top_k

In [37]:
ROOT_DIR = '/home/prasoon/multilingual-long-reasoning'
PATH_TO_FOLDER =  ROOT_DIR + '/' + 'prompts_2k'

In [42]:
top_k = 20
separator = {'en': '.', 'ar': '.', 'de': '.', 'es' : '.', 'vi': '.', 'zh' : '。', 'hi': '।'}

for file in os.listdir(PATH_TO_FOLDER):
    
    language = file.split('.')[0]
    df = pd.read_csv(PATH_TO_FOLDER + '/' + file)
    rag_prompts = []
    for index, row in df.iterrows():

        passage = row['prompt'].split('<context>')[-1].split('</context>')[0]
        sentences = passage.strip().split(separator[language])[:-1]
        query = row['prompt'].split('</context>')[-1].split('\n')[2].split('QUESTION:')[-1].strip()
        query_embedding = model.encode(query)
        passage_embeddings = model.encode(sentences)
        
        similarity = model.similarity(query_embedding, passage_embeddings)
        indices = np.argsort(similarity[0].tolist())[-top_k:][::-1]
        chosen_sentences = [sentences[index].strip() for index in indices]
        joiner = separator[language] + ' '
        chosen_passage = joiner.join(chosen_sentences)+'.'
        rag_prompt = row['prompt'].split('<context>')[0] + '<context>\n' + chosen_passage + '\n</context>' +  row['prompt'].split('</context>')[-1]
        rag_prompts.append(rag_prompt)

    df['rag_prompt_{}'.format(top_k)] = rag_prompts
    df.to_csv(PATH_TO_FOLDER + '/' + file, index=False)